# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chziha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chziha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\chziha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
import os
import re
import pickle

import pandas as pd

from sqlalchemy import create_engine

from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, accuracy_score, make_scorer

In [3]:
print(os.getcwd())

C:\Users\chziha\Data-Scientist-Udacity-Nanodegree\DSND_Term2\project_files\Disaster-Response-Pipelines-trials


In [4]:
# load data from database
engine = create_engine('sqlite:///C:\\Users\\chziha\\Data-Scientist-Udacity-Nanodegree\\DSND_Term2\\project_files\\Disaster-Response-Pipelines-trials\\trial.db')
df = pd.read_sql_table('MessageCategory', engine)
df.head() 

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df.genre.unique()

array(['direct', 'social', 'news'], dtype=object)

In [6]:
# Define feature and target variables
X = df['message']
y = df[df.columns.drop(['id', 'message', 'original', 'genre'])]

In [7]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [8]:
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [9]:
def tokenize(text):
    '''
    Function to tokenize the text data
    
    Args:
        text: a string
        
    Returns:
        words: a list of tokenized words
    '''
    # Replace all urls
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    urls = re.findall(url_regex, text)
    for url in urls:
        text.replace(url, 'urlplaceholder')
    
    # Normalize the text
    pattern = r'[^a-zA-Z0-9]'
    text = re.sub(pattern, ' ', text.lower())
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    lemmed = [lemmatizer.lemmatize(w).strip() for w in tokens]

    # Remove stop words    
    words = [w for w in lemmed if w not in stopwords.words('english')]
    
    return words

In [10]:
# Test the tokenize function
print(df.message[12])
print(tokenize(df.message[12]))

I am in Thomassin number 32, in the area named Pyron. I would like to have some water. Thank God we are fine, but we desperately need water. Thanks
['thomassin', 'number', '32', 'area', 'named', 'pyron', 'would', 'like', 'water', 'thank', 'god', 'fine', 'desperately', 'need', 'water', 'thanks']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
pipeline = Pipeline([
    ('tfidfVect', TfidfVectorizer(tokenizer=tokenize)),
    ('RFclf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 23)

In [13]:
# Train the pipeline
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidfVect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_patte...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                  

In [14]:
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [15]:
def report_metrics(y_true, y_pred, sort_by = None):
    '''
    Function to display the classification report for each category in multi-class classification
    
    Args:
        y_true: a DataFrame with true labels, each row represents the labels for one row in X
        y_pred: 2d array-like with predicted labels, each row represents the labels for one row in X
        sort_by: a string, denoting the column to sort by, select from ['Accuracy', 'F1_score', 'Precision', 'Recall']
        
    Returns:
        df: a Dataframe containing the scores for each category
    '''
    cols = y_true.columns.tolist()
    
    df_list = []
    
    for i, col in enumerate(cols):
        accuracy = accuracy_score(y_true.to_numpy()[:, i], y_pred[:, i])
        f1 = f1_score(y_true.to_numpy()[:, i], y_pred[:, i], average='weighted')
        precision = precision_score(y_true.to_numpy()[:, i], y_pred[:, i], average='weighted')
        recall = recall_score(y_true.to_numpy()[:, i], y_pred[:, i], average='weighted')
        
        df_list.append([col, accuracy, f1, precision, recall])
      
    df = pd.DataFrame(df_list, columns=['Category', 'Accuracy', 'F1_score', 'Precision', 'Recall'])
    
    if sort_by is None:
        print(df)
        return df
    
    df.sort_values(by = sort_by, axis = 0, ascending=False, inplace=True, ignore_index=True)
    print(df)
    return df

In [16]:
report_metrics(y_test, y_pred)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                  Category  Accuracy  F1_score  Precision    Recall
0                  related  0.817928  0.802527   0.805219  0.817928
1                  request  0.893961  0.880401   0.892663  0.893961
2                    offer  0.995931  0.993901   0.991879  0.995931
3              aid_related  0.782708  0.781046   0.781537  0.782708
4             medical_help  0.924221  0.896520   0.905788  0.924221
5         medical_products  0.952193  0.933215   0.944087  0.952193
6        search_and_rescue  0.972537  0.960284   0.968370  0.972537
7                 security  0.980547  0.971042   0.961719  0.980547
8                 military  0.969994  0.957451   0.965053  0.969994
9              child_alone  1.000000  1.000000   1.000000  1.000000
10                   water  0.952066  0.941584   0.949964  0.952066
11                    food  0.935029  0.927106   0.932291  0.935029
12                 shelter  0.936427  0.925147   0.930208  0.936427
13                clothing  0.986141  0.981052  

,Category,Accuracy,F1_score,Precision,Recall
0,related,0.817928,0.802527,0.805219,0.817928
1,request,0.893961,0.880401,0.892663,0.893961
2,offer,0.995931,0.993901,0.991879,0.995931
3,aid_related,0.782708,0.781046,0.781537,0.782708
4,medical_help,0.924221,0.896520,0.905788,0.924221
5,medical_products,0.952193,0.933215,0.944087,0.952193
6,search_and_rescue,0.972537,0.960284,0.968370,0.972537
7,security,0.980547,0.971042,0.961719,0.980547
8,military,0.969994,0.957451,0.965053,0.969994
9,child_alone,1.000000,1.000000,1.000000,1.000000


### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
# Check if the data is skewed
for column in y:
    print(y[column].value_counts()[0] / y.shape[0])

0.23352151357949344
0.8293408605431798
0.9954989319499542
0.5857491608178211
0.9205065608788526
0.9499160817821178
0.9723832773878548
0.9820338724443088
0.9671956057369545
1.0
0.936222154409521
0.8885032041501373
0.9117332926457126
0.9845514189807751
0.9769606347268843
0.9886328959414098
0.9666234360695758
0.9544552944766554
0.8685535550808666
0.9349633811412877
0.9541882819652121
0.9491531888922795
0.9797070491303022
0.9939350015257857
0.9892050656087885
0.9954226426609704
0.9882133048519988
0.9560955141898078
0.7216585291425084
0.9177982911199267
0.9068126335062557
0.9892432102532804
0.9063548977723528
0.9797833384192859
0.9475129691791272
0.8064159292035398


In [18]:
# Use a customized metric to score the model
def mean_f1_score(y_true, y_pred):
    '''
    Function to calculate the mean f1 score for multi-class classification
    
    Args:
        y_true: a DataFrame with true labels, each row represents the labels for one row in X
        y_pred: 2d array-like with predicted labels, each row represents the labels for one row in X
        
    Returns:
        f1_mean: a float for the mean f1 score
    '''
    f1_list = [f1_score(y_true.to_numpy()[:, i], y_pred[:, i], average='weighted') for i in range(y_true.shape[1])]
    f1_mean = sum(f1_list) / len(f1_list)
    return f1_mean

score_method = make_scorer(mean_f1_score, greater_is_better=True)

In [19]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'tfidfVect', 'RFclf', 'tfidfVect__analyzer', 'tfidfVect__binary', 'tfidfVect__decode_error', 'tfidfVect__dtype', 'tfidfVect__encoding', 'tfidfVect__input', 'tfidfVect__lowercase', 'tfidfVect__max_df', 'tfidfVect__max_features', 'tfidfVect__min_df', 'tfidfVect__ngram_range', 'tfidfVect__norm', 'tfidfVect__preprocessor', 'tfidfVect__smooth_idf', 'tfidfVect__stop_words', 'tfidfVect__strip_accents', 'tfidfVect__sublinear_tf', 'tfidfVect__token_pattern', 'tfidfVect__tokenizer', 'tfidfVect__use_idf', 'tfidfVect__vocabulary', 'RFclf__estimator__bootstrap', 'RFclf__estimator__ccp_alpha', 'RFclf__estimator__class_weight', 'RFclf__estimator__criterion', 'RFclf__estimator__max_depth', 'RFclf__estimator__max_features', 'RFclf__estimator__max_leaf_nodes', 'RFclf__estimator__max_samples', 'RFclf__estimator__min_impurity_decrease', 'RFclf__estimator__min_impurity_split', 'RFclf__estimator__min_samples_leaf', 'RFclf__estimator__min_samples_split', 'RFclf__estim

In [20]:
parameters = {
    'tfidfVect__min_df': [1, 5, 10],
    'RFclf__estimator__min_samples_leaf': [1, 2, 5]    
}

cv = GridSearchCV(pipeline, param_grid=parameters, scoring=score_method, cv=3, verbose=10)

In [21]:
# Perform grid search
model_opt = cv.fit(X_train, y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] RFclf__estimator__min_samples_leaf=1, tfidfVect__min_df=1 .......


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  RFclf__estimator__min_samples_leaf=1, tfidfVect__min_df=1, score=0.934, total= 5.4min
[CV] RFclf__estimator__min_samples_leaf=1, tfidfVect__min_df=1 .......


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  5.4min remaining:    0.0s


[CV]  RFclf__estimator__min_samples_leaf=1, tfidfVect__min_df=1, score=0.937, total= 5.4min
[CV] RFclf__estimator__min_samples_leaf=1, tfidfVect__min_df=1 .......


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed: 38.9min remaining:    0.0s


[CV] RFclf__estimator__min_samples_leaf=1, tfidfVect__min_df=10 ......
[CV]  RFclf__estimator__min_samples_leaf=1, tfidfVect__min_df=10, score=0.939, total= 4.6min


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 43.5min remaining:    0.0s


[CV] RFclf__estimator__min_samples_leaf=2, tfidfVect__min_df=1 .......
[CV]  RFclf__estimator__min_samples_leaf=2, tfidfVect__min_df=1, score=0.929, total= 3.9min
[CV] RFclf__estimator__min_samples_leaf=2, tfidfVect__min_df=1 .......
[CV]  RFclf__estimator__min_samples_leaf=2, tfidfVect__min_df=1, score=0.931, total= 3.9min
[CV] RFclf__estimator__min_samples_leaf=2, tfidfVect__min_df=1 .......
[CV]  RFclf__estimator__min_samples_leaf=2, tfidfVect__min_df=1, score=0.930, total= 3.9min
[CV] RFclf__estimator__min_samples_leaf=2, tfidfVect__min_df=5 .......
[CV]  RFclf__estimator__min_samples_leaf=2, tfidfVect__min_df=5, score=0.936, total= 3.9min
[CV] RFclf__estimator__min_samples_leaf=2, tfidfVect__min_df=5 .......
[CV]  RFclf__estimator__min_samples_leaf=2, tfidfVect__min_df=5, score=0.937, total= 3.8min
[CV] RFclf__estimator__min_samples_leaf=2, tfidfVect__min_df=5 .......
[CV]  RFclf__estimator__min_samples_leaf=2, tfidfVect__min_df=5, score=0.937, total= 3.7min
[CV] RFclf__estimator_

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed: 103.7min finished


In [22]:
# Show the summary of the grid search
model_opt.cv_results_

{'mean_fit_time': array([270.91970285, 218.24040397, 218.64519827, 178.68266273,
        175.24496198, 176.56864921, 132.5029854 , 125.47923644,
        128.0713071 ]),
 'std_fit_time': array([ 2.5882466 ,  1.92034751,  6.09204864,  1.50783342,  4.16277779,
         6.40609593, 16.30582059,  0.80179032,  1.23046626]),
 'mean_score_time': array([55.96070472, 52.56759135, 53.67125217, 55.61743553, 52.10511486,
        52.75852378, 43.30824471, 41.69639715, 41.28014437]),
 'std_score_time': array([0.22809567, 0.09200589, 1.73005303, 0.1784889 , 1.77746507,
        2.09241892, 0.58518488, 0.21235859, 0.04588102]),
 'param_RFclf__estimator__min_samples_leaf': masked_array(data=[1, 1, 1, 2, 2, 2, 5, 5, 5],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value='?',
             dtype=object),
 'param_tfidfVect__min_df': masked_array(data=[1, 5, 10, 1, 5, 10, 1, 5, 10],
              mask=[False, False, False, False, False, F

In [23]:
# Show the optimal parameters
model_opt.best_params_

{'RFclf__estimator__min_samples_leaf': 1, 'tfidfVect__min_df': 10}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [24]:
y_pred_opt = model_opt.predict(X_test)
report_metrics(y_test, y_pred_opt, sort_by='F1_score')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                  Category  Accuracy  F1_score  Precision    Recall
0              child_alone  1.000000  1.000000   1.000000  1.000000
1                    offer  0.995931  0.993901   0.991879  0.995931
2                    shops  0.995423  0.993139   0.990866  0.995423
3                    tools  0.993388  0.990094   0.986821  0.993388
4                     fire  0.991227  0.987220   0.988461  0.991227
5           missing_people  0.988811  0.983830   0.986745  0.988811
6                 clothing  0.986777  0.983546   0.983919  0.986777
7                hospitals  0.988175  0.982425   0.976741  0.988175
8              aid_centers  0.986014  0.979070   0.972224  0.986014
9                     cold  0.981437  0.975835   0.977710  0.981437
10              earthquake  0.974825  0.974404   0.974225  0.974825
11             electricity  0.981055  0.973144   0.975870  0.981055
12                security  0.980292  0.970914   0.961714  0.980292
13                   money  0.975334  0.963879  

,Category,Accuracy,F1_score,Precision,Recall
0,child_alone,1.000000,1.000000,1.000000,1.000000
1,offer,0.995931,0.993901,0.991879,0.995931
2,shops,0.995423,0.993139,0.990866,0.995423
3,tools,0.993388,0.990094,0.986821,0.993388
4,fire,0.991227,0.987220,0.988461,0.991227
5,missing_people,0.988811,0.983830,0.986745,0.988811
6,clothing,0.986777,0.983546,0.983919,0.986777
7,hospitals,0.988175,0.982425,0.976741,0.988175
8,aid_centers,0.986014,0.979070,0.972224,0.986014
9,cold,0.981437,0.975835,0.977710,0.981437


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [25]:
# Try another ML algorithms

pipeline2 = Pipeline([
    ('tfidfVect', TfidfVectorizer(tokenizer=tokenize)),
    ('RFclf', MultiOutputClassifier(SVC()))
])

In [26]:
pipeline2.get_params()

{'memory': None,
 'steps': [('tfidfVect',
   TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.float64'>, encoding='utf-8',
                   input='content', lowercase=True, max_df=1.0, max_features=None,
                   min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                   smooth_idf=True, stop_words=None, strip_accents=None,
                   sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x000001997B069C18>,
                   use_idf=True, vocabulary=None)),
  ('RFclf',
   MultiOutputClassifier(estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                                       class_weight=None, coef0=0.0,
                                       decision_function_shape='ovr', degree=3,
                                       gamma='scale', kernel='rbf', max_iter=-1,
                                       probability=False, rand

In [27]:
parameters2 = {
    'tfidfVect__min_df': [1, 5, 10],
    'tfidfVect__smooth_idf' : [True, False],
    'tfidfVect__use_idf': [True, False],
    'RFclf__estimator__C': [0.03, 0.1, 0.3 ,1, 3],
    'RFclf__estimator__kernel': ['linear', 'poly', 'rbf', 'sigmoid']
}

cv2 = GridSearchCV(pipeline2, param_grid=parameters2, scoring=score_method, verbose=10)

### 9. Export your model as a pickle file

In [28]:
# pickle.dump(model_opt, open('model_opt.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.